In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_hub as hub
import dataset as d
import random
from tqdm import tqdm

In [2]:
num_words = 1000


In [3]:
# grab the universal sentence encoder from tf hub
# embed_use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

In [4]:
data = d.load_data(1000000)
train, test = d.split_data(data)
train = d.process_data(train, 10, 100)
x, y = d.shuffle_and_split(train)


999999it [00:26, 37245.43it/s]
100%|██████████| 900000/900000 [00:23<00:00, 39068.29it/s]


In [5]:
tokenizer = d.create_tokenizer(data)
vocab_size = len(tokenizer.word_index) + 1

In [6]:
input_text = keras.Input((), dtype = tf.string, name = 'input_text')
embedding = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-large/5", input_shape = [], dtype = tf.string, trainable = False)(input_text)
dense = layers.Dense(512 * 2, activation="relu")(embedding)
dense = layers.Dropout(0.3)(dense)
dense = layers.Dense(512 * 4, activation="relu")(dense)
dense = layers.Dropout(0.2)(dense)
dense = layers.Dense(512 * 8, activation="relu")(dense)
dense = layers.Dropout(0.2)(dense)
dense = layers.Dense(512 * 8, activation="relu")(dense)
dense = layers.Dropout(0.1)(dense)
dense = layers.Dense(512 * 8, activation="relu")(dense)
dense = layers.Dropout(0.1)(dense)
dense = layers.Dense(512 * 4, activation="relu")(dense)
dense = layers.Dropout(0.05)(dense)
dense = layers.Dense(512 * 2, activation="relu")(dense)
dense = layers.Dense(512 * 1, activation="relu")(dense)
output = layers.Dense(num_words, activation="softmax")(dense)

In [7]:
model = keras.Model(inputs=input_text, outputs=output)

In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_text (InputLayer)      [(None,)]                 0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 512)               147354880 
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 4096)              839270

In [9]:
Y = []
X = []


one_hot = np.eye(num_words, dtype='int8')


for _x, _y in zip(x, y):
    if _y in tokenizer.word_index:
        if tokenizer.word_index[_y] < num_words:
      
            Y.append(one_hot[tokenizer.word_index[_y]])
            X.append(_x)
        
        
Y = np.array(Y)
X = np.array(X)

In [10]:
len(X)

1360038

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(x=X, y=Y, batch_size=256, epochs=5, validation_split=0.02)

Epoch 1/5
 883/5207 [====>.........................] - ETA: 11:06 - loss: 5.6590 - accuracy: 0.0977

In [63]:
tokenizer.word_index['the']

1

In [98]:
a = model.predict(['The office is'])

In [99]:
tokenizer.index_word[tf.argmax(tf.reshape(a, (-1))).numpy()]

'well'

In [100]:
import pickle
model.save("model_data/model.h5")


In [101]:
with open('model_data/corpus.pickle', 'wb') as corpus:
    pickle.dump([tokenizer.word_index, tokenizer.index_word], corpus)

ValueError: No model found in config file.